In [1]:
import numpy as np 
import pandas as pd
import folium
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
!conda install -c conda-forge lxml --yes
import requests
import lxml.html as lh
from sklearn.cluster import KMeans
print("Libraries imported.")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)

doc = lh.fromstring(page.content)

tr_elements = doc.xpath('//tr')
r = requests.get(url)

col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

size_of_col = 3
for j in range(1,len(tr_elements)):
    
    T=tr_elements[j]
    
     
    if len(T)!=size_of_col:
        break
    
    
    i=0
    
    
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df = df.replace(r'\n','', regex=True) 
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.drop(df.tail(1).index,inplace=True)

df = df[df.Borough != "Not assigned"].reset_index(drop=True)

df = df.replace(r'/',', ', regex=True) 

for index, row in df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
df.head()

1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
df["Latitude"] = ""
df["Longitude"] = ""
df.shape

(103, 5)

In [4]:
df["Neighborhood"] = df["Neighborhood"].str.split(",", n = 1, expand = True) 
df["Neighborhood"] = df["Neighborhood"].str.split("-", n = 1, expand = True) 
df["Neighborhood"].head(5)

0           Parkwoods
1    Victoria Village
2         Regent Park
3      Lawrence Manor
4        Queen's Park
Name: Neighborhood, dtype: object

In [5]:
df.head(50)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,,
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,Regent Park,,
3,M6A,North York,Lawrence Manor,,
4,M7A,Downtown Toronto,Queen's Park,,
5,M9A,Etobicoke,Islington Avenue,,
6,M1B,Scarborough,Malvern,,
7,M3B,North York,Don Mills,,
8,M4B,East York,Parkview Hill,,
9,M5B,Downtown Toronto,Garden District,,


In [6]:
df1 = df.loc[0:25]
df2 = df.loc[26:50]
df3 = df.loc[51:75]
df4 = df.loc[76:102]

In [7]:
to_drop_unknown = []
geolocator = Nominatim(user_agent="ny_explorer")
for index, row in df1.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df1.loc[index, 'Latitude'] = latitude
        df1.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Parkwoods, Toronto are 43.7587999, -79.3201966.


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


The geograpical coordinate of Victoria Village, Toronto are 43.732658, -79.3111892.
The geograpical coordinate of Regent Park, Toronto are 43.6607056, -79.3604569.
The geograpical coordinate of Lawrence Manor, Toronto are 43.7220788, -79.4375067.
The geograpical coordinate of Queen's Park, Toronto are 43.659659, -79.3903399.
The geograpical coordinate of Islington Avenue, Toronto are 43.6389593, -79.5210499.
The geograpical coordinate of Malvern, Toronto are 43.8091955, -79.2217008.
The geograpical coordinate of Don Mills, Toronto are 43.775347, -79.3459439.
The geograpical coordinate of Parkview Hill, Toronto are 43.7062977, -79.3219073.
The geograpical coordinate of Garden District, Toronto are 43.6564995, -79.3771141.
The geograpical coordinate of Glencairn, Toronto are 43.7087117, -79.4406853.
The geograpical coordinate of West Deane Park, Toronto are 43.6631995, -79.5685684.
The geograpical coordinate of Rouge Hill, Toronto are 43.7802711, -79.1304992.
The geograpical coordinate o

In [8]:
for index, row in df2.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df2.loc[index, 'Latitude'] = latitude
        df2.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Cedarbrae, Toronto are 43.75646655, -79.22669244258802.
The geograpical coordinate of Hillcrest Village, Toronto are 43.6816953, -79.4257118.
The geograpical coordinate of Bathurst Manor, Toronto are 43.6655189, -79.4119373.
The geograpical coordinate of Thorncliffe Park, Toronto are 43.704553, -79.3454074.
The geograpical coordinate of Richmond, Toronto are 43.6485875, -79.3913729.
The geograpical coordinate of Dufferin, Toronto are 43.6602019, -79.4357191.
The geograpical coordinate of Scarborough Village, Toronto are 43.7437422, -79.2116324.
The geograpical coordinate of Fairview, Toronto are 43.777758500000004, -79.34429375180316.
The geograpical coordinate of Northwood Park, Toronto are 43.7541351, -79.50448.
The geograpical coordinate of East Toronto, Toronto are 43.626243, -79.396962.
The geograpical coordinate of Harbourfront East, Toronto are 43.6400801, -79.3801495.
The geograpical coordinate of Little Portugal, Toronto are 43.64741325, -79.43111

In [9]:
geolocator = Nominatim(user_agent="ny_explorer3")
for index, row in df3.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df3.loc[index, 'Latitude'] = latitude
        df3.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

The geograpical coordinate of Cliffside, Toronto are 43.7111699, -79.2481769.
The geograpical coordinate of Willowdale, Toronto are 43.7615095, -79.4109234.
The geograpical coordinate of Downsview, Toronto are 43.7492988, -79.462248.
The geograpical coordinate of Studio District, Toronto are 43.64958515, -79.39068322559326.
The geograpical coordinate of Bedford Park, Toronto are 43.7373876, -79.4109253.
Cannot do: Del Ray, Toronto, will drop index: 56
The geograpical coordinate of Humberlea, Toronto are 43.7213166, -79.5331605.
The geograpical coordinate of Birch Cliff, Toronto are 43.6918051, -79.2644935.
The geograpical coordinate of Willowdale, Toronto are 43.7615095, -79.4109234.
The geograpical coordinate of Downsview, Toronto are 43.7492988, -79.462248.
The geograpical coordinate of Lawrence Park, Toronto are 43.729199, -79.4032525.
The geograpical coordinate of Roselawn, Toronto are 43.7087103, -79.4097894.
The geograpical coordinate of Runnymede, Toronto are 43.6517026, -79.475

In [10]:
geolocator = Nominatim(user_agent="ny_explorer4")
for index, row in df4.iterrows():
    address = row['Neighborhood'] + ', Toronto'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df4.loc[index, 'Latitude'] = latitude
        df4.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: Canada Post Gateway Processing Centre, Toronto, will drop index: 76
The geograpical coordinate of Kingsview Village, Toronto are 43.6995391, -79.5563459.
The geograpical coordinate of Agincourt, Toronto are 43.7853531, -79.2785494.
The geograpical coordinate of Davisville, Toronto are 43.697936, -79.3972908.
The geograpical coordinate of University of Toronto, Toronto are 43.663461999999996, -79.39775965337452.
The geograpical coordinate of Runnymede, Toronto are 43.6517026, -79.4759978.
The geograpical coordinate of Clarks Corners, Toronto are 43.7964095, -79.2977951.
The geograpical coordinate of Moore Park, Toronto are 43.6903876, -79.3832965.
The geograpical coordinate of Kensington Market, Toronto are 43.6552136, -79.4022604.
The geograpical coordinate of Milliken, Toronto are 43.8231743, -79.3017626.
The geograpical coordinate of Summerhill West, Toronto are 43.6816776, -79.3905037.
The geograpical coordinate of CN Tower, Toronto are 43.6425637, -79.38708718320467.
The

In [11]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(26, 5)
(25, 5)
(25, 5)
(27, 5)


In [12]:
df = df1.append(df2, ignore_index = True)
df = df.append(df3, ignore_index = True)
df = df.append(df4, ignore_index = True)

In [13]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7588,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
3,M6A,North York,Lawrence Manor,43.722079,-79.437507
4,M7A,Downtown Toronto,Queen's Park,43.659659,-79.39034


In [14]:
clean_df = df.drop(to_drop_unknown)
clean_df['Latitude'].replace('', np.nan, inplace=True)
clean_df.dropna(subset=['Latitude'], inplace=True)
clean_df.shape

(99, 5)

In [15]:
clean_df.tail()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
97,M5X,Downtown Toronto,First Canadian Place,43.648768,-79.381692
98,M8X,Etobicoke,The Kingsway,43.647381,-79.511333
99,M4Y,Downtown Toronto,Church and Wellesley,43.670862,-79.372792
101,M8Y,Etobicoke,Old Mill South,43.649826,-79.494334
102,M8Z,Etobicoke,Mimico NW,43.616677,-79.496805


In [16]:
Toronto_df = clean_df[clean_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
Toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
1,M7A,Downtown Toronto,Queen's Park,43.659659,-79.39034
2,M5B,Downtown Toronto,Garden District,43.6565,-79.377114
3,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
4,M4E,East Toronto,The Beaches,43.671024,-79.296712


In [17]:
Toronto_df.drop_duplicates(subset ="PostalCode",keep = "first", inplace = True)
Toronto_df.reset_index(drop=True) 
print(Toronto_df.shape)

(37, 5)


In [18]:
CLIENT_ID = 'WTS0CPG4ZLPA5XWFKYZW5E0Z5SOMIKVKMXZFH2ZB3OVMYV54' 
CLIENT_SECRET = '3CNPECGO1WRHEBAYXNPWPTPR2O4PRLCZ0D4IKYGZLSPYLKVA' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
toronto_df_new = Toronto_df.copy()

Your credentails:
CLIENT_ID: WTS0CPG4ZLPA5XWFKYZW5E0Z5SOMIKVKMXZFH2ZB3OVMYV54
CLIENT_SECRET:3CNPECGO1WRHEBAYXNPWPTPR2O4PRLCZ0D4IKYGZLSPYLKVA


In [19]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,        CLIENT_SECRET,        VERSION,        lat,        long,        radius,         LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post,             borough,            neighborhood,            lat,             long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1865, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Sumach Espresso,43.658135,-79.359515,Coffee Shop
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
3,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Thai To Go,43.663418,-79.360710,Thai Restaurant
4,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Paintbox Bistro,43.660050,-79.362855,Restaurant


In [21]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count().head()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,44,44,44,44,44,44
M4K,East Toronto,The Danforth West,30,30,30,30,30,30
M4L,East Toronto,India Bazaar,35,35,35,35,35,35
M4M,East Toronto,Studio District,100,100,100,100,100,100
M4N,Central Toronto,Lawrence Park,51,51,51,51,51,51


In [22]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 240 uniques categories.


In [23]:
venues_df['VenueCategory'].unique()[:10]

array(['Pool', 'Coffee Shop', 'Performing Arts Venue', 'Thai Restaurant',
       'Restaurant', 'Pub', 'Animal Shelter', 'Sushi Restaurant',
       'Auto Dealership', 'Food Truck'], dtype=object)

In [24]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1865, 243)


,PostalCode,Borough,Neighborhoods,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(37, 243)


,PostalCode,Borough,Neighborhoods,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.045455,0.00
1,M4K,East Toronto,The Danforth West,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
2,M4L,East Toronto,India Bazaar,0.0,0.0,0.0,0.0,0.0,0.0,0.028571,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,...,0.01,0.04,0.01,0.0,0.0,0.0,0.0,0.0,0.000000,0.02
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00


In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(37, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Beach,Women's Store,Pub,Japanese Restaurant,Breakfast Spot,Pizza Place,Park,Bar,Pharmacy,Mediterranean Restaurant
1,M4K,East Toronto,The Danforth West,Pharmacy,Bus Line,Coffee Shop,Grocery Store,Skating Rink,Fried Chicken Joint,Caribbean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
2,M4L,East Toronto,India Bazaar,Indian Restaurant,Grocery Store,Café,Donut Shop,Bar,Shopping Plaza,Snack Place,Bus Stop,Diner,Brewery
3,M4M,East Toronto,Studio District,Coffee Shop,Vegetarian / Vegan Restaurant,Café,Clothing Store,Shoe Store,Cosmetics Shop,Japanese Restaurant,Hotel,Italian Restaurant,Event Space
4,M4N,Central Toronto,Lawrence Park,Italian Restaurant,Sushi Restaurant,Bakery,Coffee Shop,Cosmetics Shop,Pub,Fast Food Restaurant,Burger Joint,Bank,Pharmacy


In [27]:
kclusters = 5

Toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 3, 1, 3, 4, 3], dtype=int32)

In [28]:
print(toronto_grouped.shape)
print(neighborhoods_venues_sorted.shape)
print(toronto_df_new.shape)
print(kmeans.labels_.size)

(37, 243)
(37, 13)
(37, 5)
37


In [29]:
toronto_merged = toronto_df_new.copy()

toronto_merged["Cluster Labels"] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")
toronto_merged.rename(columns = {'BoroughLatitude':'Latitude', 'BoroughLongitude':'Longitude'}, inplace = True) 
print(toronto_merged.shape)
toronto_merged.head() 

(37, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,3,Coffee Shop,Thai Restaurant,Restaurant,Indian Restaurant,Auto Dealership,Sushi Restaurant,Grocery Store,Pub,Electronics Store,Fast Food Restaurant
1,M7A,Downtown Toronto,Queen's Park,43.659659,-79.39034,3,Coffee Shop,Café,Sandwich Place,Italian Restaurant,French Restaurant,Japanese Restaurant,Bubble Tea Shop,Thai Restaurant,Restaurant,Bank
2,M5B,Downtown Toronto,Garden District,43.6565,-79.377114,3,Clothing Store,Hotel,Coffee Shop,Restaurant,Movie Theater,Café,Sandwich Place,Cosmetics Shop,Theater,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704,3,Coffee Shop,Pizza Place,Bakery,Café,Grocery Store,Restaurant,Intersection,Filipino Restaurant,Food & Drink Shop,Gym / Fitness Center
4,M4E,East Toronto,The Beaches,43.671024,-79.296712,3,Beach,Women's Store,Pub,Japanese Restaurant,Breakfast Spot,Pizza Place,Park,Bar,Pharmacy,Mediterranean Restaurant


In [30]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged.head()

(37, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,M4Y,Downtown Toronto,Church and Wellesley,43.670862,-79.372792,0,Coffee Shop,Pizza Place,Grocery Store,Hotel,Bakery,Metro Station,Sandwich Place,Bistro,Bike Rental / Bike Share,Market
21,M5P,Central Toronto,Forest Hill North & West,43.701028,-79.4256,0,Pizza Place,Bagel Shop,Burger Joint,Trail,Pharmacy,Optical Shop,Restaurant,Coffee Shop,Sandwich Place,Chinese Restaurant
20,M4P,Central Toronto,Davisville North,43.697936,-79.397291,0,Sushi Restaurant,Italian Restaurant,Coffee Shop,Park,Ice Cream Shop,Pub,Convenience Store,Spa,Business Service,Café
19,M5N,Central Toronto,Roselawn,43.70871,-79.409789,0,Restaurant,Park,Liquor Store,Sushi Restaurant,Tea Room,Gym Pool,Japanese Restaurant,Intersection,Deli / Bodega,Fast Food Restaurant
35,M5X,Downtown Toronto,First Canadian Place,43.648768,-79.381692,0,Coffee Shop,Hotel,Restaurant,Café,Gym,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant


In [31]:
map_clusters  = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
rainbow = [    'red',    'blue',    'orange',    'darkgreen',    'darkblue',    'black']

markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/RohitLearner/IBM-Applied-D.S-Capstone-Project/master/Week%203/Results/Toronto_Clustered.png")

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Downtown Toronto,0,Coffee Shop,Pizza Place,Grocery Store,Hotel,Bakery,Metro Station,Sandwich Place,Bistro,Bike Rental / Bike Share,Market
21,Central Toronto,0,Pizza Place,Bagel Shop,Burger Joint,Trail,Pharmacy,Optical Shop,Restaurant,Coffee Shop,Sandwich Place,Chinese Restaurant
20,Central Toronto,0,Sushi Restaurant,Italian Restaurant,Coffee Shop,Park,Ice Cream Shop,Pub,Convenience Store,Spa,Business Service,Café
19,Central Toronto,0,Restaurant,Park,Liquor Store,Sushi Restaurant,Tea Room,Gym Pool,Japanese Restaurant,Intersection,Deli / Bodega,Fast Food Restaurant
35,Downtown Toronto,0,Coffee Shop,Hotel,Restaurant,Café,Gym,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant
17,East Toronto,0,Coffee Shop,Vegetarian / Vegan Restaurant,Café,Clothing Store,Shoe Store,Cosmetics Shop,Japanese Restaurant,Hotel,Italian Restaurant,Event Space
16,Downtown Toronto,0,Coffee Shop,Hotel,Restaurant,Seafood Restaurant,Café,Gastropub,Gym,Italian Restaurant,Japanese Restaurant,Beer Bar
15,East Toronto,0,Indian Restaurant,Grocery Store,Café,Donut Shop,Bar,Shopping Plaza,Snack Place,Bus Stop,Diner,Brewery
13,Downtown Toronto,0,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Seafood Restaurant,Asian Restaurant,Japanese Restaurant,Gastropub,Beer Bar
12,East Toronto,0,Pharmacy,Bus Line,Coffee Shop,Grocery Store,Skating Rink,Fried Chicken Joint,Caribbean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,West Toronto,1,Tibetan Restaurant,Pharmacy,Diner,Restaurant,Indian Restaurant,Pizza Place,Bakery,Chinese Restaurant,Hawaiian Restaurant,Boutique
26,Central Toronto,1,Sushi Restaurant,Italian Restaurant,Coffee Shop,Park,Ice Cream Shop,Pub,Convenience Store,Spa,Business Service,Café
29,Central Toronto,1,Convenience Store,Restaurant,Tennis Court,Gym,Nail Salon,Nightclub,New American Restaurant,Neighborhood,Music Venue,North Indian Restaurant
6,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Middle Eastern Restaurant,Burger Joint,Chinese Restaurant,Bubble Tea Shop,Gastropub,Comic Shop
30,Downtown Toronto,1,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Bakery,Farmers Market,Breakfast Spot,Sandwich Place
31,Central Toronto,1,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Spa,Pharmacy,Beer Store,Gym / Fitness Center,Convenience Store,Grocery Store
32,Downtown Toronto,1,Hotel,Pizza Place,Coffee Shop,Baseball Stadium,Scenic Lookout,Gym,Ice Cream Shop,Bar,Aquarium,Supermarket


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Pizza Place,Music Venue,Chinese Restaurant,Sandwich Place,Sporting Goods Shop


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,3,Café,Japanese Restaurant,Park,Bookstore,French Restaurant,College Arts Building,College Gym,Comfort Food Restaurant,Dessert Shop,Hotel
33,Downtown Toronto,3,Park,Construction & Landscaping,Playground,Bike Trail,Noodle House,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater
34,Downtown Toronto,3,Coffee Shop,Pizza Place,Bakery,Café,Grocery Store,Restaurant,Intersection,Filipino Restaurant,Food & Drink Shop,Gym / Fitness Center
24,Central Toronto,3,Pizza Place,Bistro,Grocery Store,Thai Restaurant,Pharmacy,Burrito Place,Sandwich Place,Coffee Shop,Ramen Restaurant,Pub
0,Downtown Toronto,3,Coffee Shop,Thai Restaurant,Restaurant,Indian Restaurant,Auto Dealership,Sushi Restaurant,Grocery Store,Pub,Electronics Store,Fast Food Restaurant
22,West Toronto,3,Pizza Place,Gym,Pharmacy,Pet Store,Pool,Mexican Restaurant,Tennis Court,Metro Station,Convenience Store,Opera House
14,West Toronto,3,Bar,Park,Vietnamese Restaurant,Pizza Place,Café,French Restaurant,Gastropub,Coffee Shop,Dive Bar,Bakery
9,West Toronto,3,Bar,Bakery,Café,Restaurant,Coffee Shop,Mexican Restaurant,Sandwich Place,Cocktail Bar,Vietnamese Restaurant,Beer Store
7,Downtown Toronto,3,Korean Restaurant,Coffee Shop,Grocery Store,Mexican Restaurant,Café,Indian Restaurant,Pub,Ice Cream Shop,Cocktail Bar,Dessert Shop
5,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Cocktail Bar,Hotel,Gastropub,Breakfast Spot


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,4,Coffee Shop,Hotel,Café,Clothing Store,Thai Restaurant,Shoe Store,Restaurant,Indian Restaurant,Concert Hall,Cosmetics Shop
